In [ ]:
import pandas as pd
import shap
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from explainbench.shap_wrapper import SHAPExplainer
from explainbench.lime_wrapper import LIMEExplainer
from explainbench.counterfactuals import CounterfactualExplainer

# Load COMPAS dataset (assumed to be cleaned)
df = pd.read_csv('../datasets/compas_clean.csv')

# Preprocess data
target = 'two_year_recid'
categorical = ['sex', 'race', 'age_cat', 'c_charge_degree']

for col in categorical:
    df[col] = LabelEncoder().fit_transform(df[col])

X = df.drop(columns=[target])
y = df[target]

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predictions for fairness analysis
y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)[:, 1]
X_test['pred'] = y_pred
X_test['actual'] = y_test.values
X_test['prob'] = y_prob
X_test['race'] = df.loc[X_test.index, 'race']
X_test['sex'] = df.loc[X_test.index, 'sex']

# Accuracy by group
def group_accuracy(df, group_col):
    return df.groupby(group_col).apply(lambda x: accuracy_score(x['actual'], x['pred'])).rename('accuracy')

print("Accuracy by race:")
print(group_accuracy(X_test, 'race'))

print("Accuracy by sex:")
print(group_accuracy(X_test, 'sex'))

# Disparate impact ratio
def disparate_impact(df, protected_col, privileged_val):
    favorable = df['pred'] == 0
    groups = df.groupby(protected_col)
    prob_priv = (groups.get_group(privileged_val)['pred'] == 0).mean()
    ratios = {}
    for group, group_df in groups:
        if group != privileged_val:
            prob_group = (group_df['pred'] == 0).mean()
            ratios[group] = round(prob_group / prob_priv, 3)
    return ratios

print("Disparate impact by race (privileged=0):")
print(disparate_impact(X_test, 'race', 0))

print("Disparate impact by sex (privileged=0):")
print(disparate_impact(X_test, 'sex', 0))

# SHAP explanations
explainer = SHAPExplainer(model, X_train, model_type='tree')
shap_values = explainer.explain_global(num_samples=200)
explainer.plot_summary(shap_values, features=X_train)

# Save SHAP plot
plt.figure()
shap.summary_plot(shap_values.values, X_train, show=False)
plt.tight_layout()
plt.savefig('shap_summary_plot.png')

# LIME demo
lime_explainer = LIMEExplainer(model, X_train)
instance = X_test.drop(columns=['pred', 'actual', 'prob', 'race', 'sex']).iloc[0]
lime_exp = lime_explainer.explain_instance(instance)
lime_exp.show_in_notebook()

# Counterfactual demo (if dice-ml is installed)
try:
    cf_explainer = CounterfactualExplainer(model, X_train, y_train)
    cf = cf_explainer.generate(instance)
    print("Counterfactual examples:")
    print(cf.cf_examples_list[0].final_cfs_df)
except ImportError:
    print("dice-ml not installed. Skipping counterfactual demo.")


Income dataset

In [ ]:
import pandas as pd
import shap
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from explainbench.shap_wrapper import SHAPExplainer
from explainbench.lime_wrapper import LIMEExplainer
from explainbench.counterfactuals import CounterfactualExplainer

# Load UCI Adult dataset
df = pd.read_csv('../datasets/adult.csv')

# Preprocess data
target = 'income'
categorical = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']

for col in categorical:
    df[col] = LabelEncoder().fit_transform(df[col])

df[target] = LabelEncoder().fit_transform(df[target])

X = df.drop(columns=[target])
y = df[target]

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# SHAP explanations
explainer = SHAPExplainer(model, X_train, model_type='tree')
shap_values = explainer.explain_global(num_samples=200)
explainer.plot_summary(shap_values, features=X_train)

# Save SHAP plot
plt.figure()
shap.summary_plot(shap_values.values, X_train, show=False)
plt.tight_layout()
plt.savefig('adult_shap_summary_plot.png')

# LIME explanation
lime_explainer = LIMEExplainer(model, X_train)
instance = X_test.iloc[0]
lime_exp = lime_explainer.explain_instance(instance)
lime_exp.show_in_notebook()

# Counterfactuals (optional)
try:
    cf_explainer = CounterfactualExplainer(model, X_train, y_train)
    cf = cf_explainer.generate(instance)
    print("Counterfactual examples:")
    print(cf.cf_examples_list[0].final_cfs_df)
except ImportError:
    print("dice-ml not installed. Skipping counterfactual demo.")
